In [44]:
import numpy as np
import pandas as pd
import os,sys,inspect
import scipy.stats as stats
import scipy.optimize as opt
from scipy.stats import norm
import scipy.integrate as integrate
from scipy.io import loadmat
import econtools 
import econtools.metrics as mt
import statsmodels.discrete.discrete_model as sm
import matplotlib.pyplot as plt
import itertools as it
import copy
import importlib
import warnings
warnings.filterwarnings('ignore')



# my functions
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
#targetdir = os.path.dirname(currentdir)
targetdir = currentdir

if targetdir not in sys.path:
    sys.path.insert(0,targetdir)
    
from entry import entry_Q1
from entry import entry_Q2

importlib.reload(entry_Q1)
importlib.reload(entry_Q2)

<module 'entry.entry_Q2' from '/Users/jingyuanwang/GitHub/NU450_HW/450-2-HW1/entry/entry_Q2.py'>

In [2]:
# ------------------------------------------------------------------------
# NOTE
# ------------------------------------------------------------------------
# Purpose: HW1 Q1 
# 
# ------------------------------------------------------------------------

# path

In [3]:
# 0. Initialize file path -----------------------------------------------
rootpath = '/Users/jingyuanwang/Dropbox/Course/ECON/IO/NU450/NU450_HW/450-2-HW1'
datapath = rootpath + '/' + 'data'
resultpath = rootpath + '/' + 'results'

# read data

In [4]:
# Q1
filename = 'entry.mat'
file = datapath + '/'+ filename
data = loadmat(file)

In [5]:
df = (pd.DataFrame(np.concatenate( 
                              (data['mrkts'],
                               data['costs'],
                               data['firms'], 
                               data['entry']), axis = 1 ))
              .rename(columns = {0:'X_m',1:'N_m',2:'pi_1m',3:'pi_2m', 4:'pi_3m', 
                                 5:'phi_1m', 6:'phi_2m', 7:'phi_3m',
                                 8:'Z_1m', 9:'Z_2m', 10:'Z_3m',
                                 11:'entry_1m', 12:'entry_2m', 13:'entry_3m'}) )

In [6]:
# Q2
filename = 'entry2.mat'
file = datapath + '/'+ filename
data = loadmat(file)

In [50]:
df_2 = (pd.DataFrame(np.concatenate( 
                              (data['mrkts'],
                               data['costs'],
                               data['firms'], 
                               data['entry']), axis = 1 ))
              .rename(columns = {0:'X_m',1:'N_m',2:'N_potential_m', 3:'xi_m',
                                 4:'pi_1m',5:'pi_2m', 6:'pi_3m', 7:'pi_4m', 8:'pi_5m', 9:'pi_6m', 10:'pi_7m',
                                 11:'phi_1m', 12:'phi_2m', 13:'phi_3m', 14:'phi_4m', 15:'phi_5m', 16:'phi_6m', 17:'phi_7m',
                                 18:'Z_1m', 19:'Z_2m', 20:'Z_3m', 21:'Z_4m', 22:'Z_5m', 23:'Z_6m', 24:'Z_7m',
                                 25:'entry_1m', 26:'entry_2m', 27:'entry_3m',
                                 28:'entry_4m', 29:'entry_5m', 30:'entry_6m', 31:'entry_7m'}) 
     )

# Q1

In [43]:
markets = entry_Q1.entry_likelihood(M=100, F=3, df=df, n_sample=10000 )

In [10]:
def obj(parameter, order=None):
    
    mu = parameter[0]
    sigma = parameter[1]
    likelihood = markets.get_likelihood(alpha=1, beta=1, delta=1, mu=mu, sigma=sigma, order = order)
    
    return -likelihood

In [37]:
# test the true value:
true_value = np.array([2,1])
print(obj(true_value))
print(obj(true_value,order = 'lowestfirst'))
print(obj(true_value,order = 'highestfirst'))

117.42332341576864
122.42196056030261
11556.192235097686


In [31]:
initial = np.array([1,2])
results0 = opt.minimize(obj, initial, args = None, method = 'Nelder-Mead')
results1 = opt.minimize(obj, initial, args = 'lowestfirst', method = 'Nelder-Mead')
results2 = opt.minimize(obj, initial, args = 'highestfirst', method = 'Nelder-Mead')

In [41]:
print('not constrained:        estimated mu, sigma = {}, obj fun = {} '.format(results0.x, results0.fun))
print('low cost enter first:   estimated mu, sigma = {}, obj fun = {} '.format(results1.x, results1.fun))
print('hight cost enter first: estimated mu, sigma = {}, obj fun = {} '.format(results2.x, results2.fun))

not constrained:        estimated mu, sigma = [2.18029756 0.96010509], obj fun = 115.62540192988874 
low cost enter first:   estimated mu, sigma = [2.14440378 1.06047752], obj fun = 120.723315041002 
hight cost enter first: estimated mu, sigma = [1.14319475 1.76663428], obj fun = 13068.4468692048 


# Q2

In [54]:
importlib.reload(entry_Q2)

<module 'entry.entry_Q2' from '/Users/jingyuanwang/GitHub/NU450_HW/450-2-HW1/entry/entry_Q2.py'>

In [55]:
markets = entry_Q2.entry_likelihood(M=100, F=11, df=df_2, n_sample=1000 )

In [53]:
df_2.head()

,X_m,N_m,N_potential_m,xi_m,pi_1m,pi_2m,pi_3m,pi_4m,pi_5m,pi_6m,...,Z_5m,Z_6m,Z_7m,entry_1m,entry_2m,entry_3m,entry_4m,entry_5m,entry_6m,entry_7m
0,4.388989,4.0,4.0,8.139496,11.654469,10.482474,9.805210,6.970965,0.000000,0.0,...,0.000000,0.000000,0.000000,1.0,1.0,1.0,1.0,0.0,0.0,0.0
1,2.145807,2.0,6.0,1.365974,1.061698,0.764171,0.000000,0.000000,0.000000,0.0,...,0.514764,0.343570,0.000000,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2.134941,2.0,7.0,0.560701,3.100604,0.484267,0.000000,0.000000,0.000000,0.0,...,1.296344,-0.394147,1.620373,1.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3.596548,3.0,5.0,0.520023,2.482523,0.994207,0.586661,0.000000,0.000000,0.0,...,1.033741,0.000000,0.000000,1.0,1.0,1.0,0.0,0.0,0.0,0.0
4,4.375764,5.0,5.0,15.479796,17.567524,16.626829,16.449622,15.594684,15.096972,0.0,...,-0.134980,0.000000,0.000000,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [58]:
controls = [x for x in df_2.columns if x.startswith('xi')]
mt.reg(df_2, 'pi_1m', ['N_m', 'X_m'] , nocons = False )

Dependent variable:	pi_1m
N:			100
R-squared:		0.1660
Estimation method:	OLS
VCE method:		Standard (Homosk.)
    coeff    se     t   p>t CI_low CI_high
N_m 0.381 0.218 1.749 0.083 -0.051   0.814
X_m 0.773 0.263 2.940 0.004  0.251   1.295

# Appendix & draft